In [ ]:
# 랜덤 x 
# [0:30000], [30000:60000], [60000:90000]
# 총 3개 모델을 Ensemble
# Soft Voting

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
import tensorflow as tf
from keras.models import Model
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split 

import unicodedata
import re
import numpy as np
import os
import io
import time

In [4]:
# 데이터 로드
path_to_file = '/content/drive/MyDrive/Colab Notebooks/spa-eng/spa.txt'
print(path_to_file)

/content/drive/MyDrive/Colab Notebooks/spa-eng/spa.txt


In [5]:
# 유니코드 파일을 아스키 코드 파일로 변환합니다.
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
  w = unicode_to_ascii(w.lower().strip())

  # 단어와 단어 뒤에 오는 구두점(.)사이에 공백을 생성합니다.
  # 예시: "he is a boy." => "he is a boy ."
  # 참고:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # (a-z, A-Z, ".", "?", "!", ",")을 제외한 모든 것을 공백으로 대체합니다.
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

  w = w.strip()

  # 모델이 예측을 시작하거나 중단할 때를 알게 하기 위해서
  # 문장에 start와 end 토큰을 추가합니다.
  w = '<start> ' + w + ' <end>'
  return w

In [6]:
# 1. 문장에 있는 억양을 제거합니다.
# 2. 불필요한 문자를 제거하여 문장을 정리합니다.
# 3. 다음과 같은 형식으로 문장의 쌍을 반환합니다: [영어, 스페인어]
def create_dataset(path, num_examples, range):
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

  word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[(num_examples*range):(num_examples*(range+1))]]

  return zip(*word_pairs)

In [7]:
# language 가 들어오면 공백 단위로 토큰화
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)

  # texts_to_sequqences : 텍스트 안의 단어들을 숫자 시퀀스로 출력
  tensor = lang_tokenizer.texts_to_sequences(lang)
    
  # 서로 다른 개수의 단어로 이루어진 문장을 같은 길이로 만들어주기 위해 패딩을 사용
  # post : 뒤에 패딩이 채워짐 (앞에서부터는 'pre')
  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

In [8]:
def load_dataset(path, range, num_examples=None):
  # 전처리된 타겟 문장과 입력 문장 쌍을 생성합니다.
  targ_lang, inp_lang = create_dataset(path, num_examples, range)

  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [9]:
# 언어 데이터셋을 아래의 크기로 제한하여 훈련과 검증을 수행합니다.
num_examples = 30000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, 0, num_examples)
input_tensor2, target_tensor2, inp_lang2, targ_lang2 = load_dataset(path_to_file, 1, num_examples)
input_tensor3, target_tensor3, inp_lang3, targ_lang3 = load_dataset(path_to_file, 2, num_examples)


# 타겟 텐서와 입력 텐서의 최대 길이를 계산합니다.
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]
max_length_targ2, max_length_inp2 = target_tensor2.shape[1], input_tensor2.shape[1]
max_length_targ3, max_length_inp3 = target_tensor3.shape[1], input_tensor3.shape[1]

# print(max_length_targ, max_length_inp)
# print(max_length_targ2, max_length_inp2)
# print(max_length_targ3, max_length_inp3)

In [10]:
# 인덱스 -> 해당 word 로.
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

In [11]:
# BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
# steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

vocab_inp_size2 = len(inp_lang2.word_index)+1
vocab_tar_size2 = len(targ_lang2.word_index)+1

vocab_inp_size3 = len(inp_lang3.word_index)+1
vocab_tar_size3 = len(targ_lang3.word_index)+1

In [12]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [13]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
encoder2 = Encoder(vocab_inp_size2, embedding_dim, units, BATCH_SIZE)
encoder3 = Encoder(vocab_inp_size3, embedding_dim, units, BATCH_SIZE)

In [14]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # 쿼리 은닉 상태(query hidden state)는 (batch_size, hidden size)쌍으로 이루어져 있습니다.
    # query_with_time_axis은 (batch_size, 1, hidden size)쌍으로 이루어져 있습니다.
    # values는 (batch_size, max_len, hidden size)쌍으로 이루어져 있습니다.
    # 스코어(score)계산을 위해 덧셈을 수행하고자 시간 축을 확장하여 아래의 과정을 수행합니다.
    query_with_time_axis = tf.expand_dims(query, 1)

    # score는 (batch_size, max_length, 1)쌍으로 이루어져 있습니다.
    # score를 self.V에 적용하기 때문에 마지막 축에 1을 얻습니다.
    # self.V에 적용하기 전에 텐서는 (batch_size, max_length, units)쌍으로 이루어져 있습니다.
    score = self.V(tf.nn.tanh(
        self.W1(query_with_time_axis) + self.W2(values)))

    # attention_weights는 (batch_size, max_length, 1)쌍으로 이루어져 있습니다. 
    attention_weights = tf.nn.softmax(score, axis=1)

    # 덧셈이후 컨텍스트 벡터(context_vector)는 (batch_size, hidden_size)쌍으로 이루어져 있습니다.
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [15]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # 어텐션을 사용합니다.
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output는 (batch_size, max_length, hidden_size)쌍으로 이루어져 있습니다.
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # 임베딩층을 통과한 후 x는 (batch_size, 1, embedding_dim)쌍으로 이루어져 있습니다.
    x = self.embedding(x)

    # 컨텍스트 벡터와 임베딩 결과를 결합한 이후 x의 형태는 (batch_size, 1, embedding_dim + hidden_size)쌍으로 이루어져 있습니다.
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # 위에서 결합된 벡터를 GRU에 전달합니다.
    output, state = self.gru(x)

    # output은 (batch_size * 1, hidden_size)쌍으로 이루어져 있습니다.
    output = tf.reshape(output, (-1, output.shape[2]))

    # output은 (batch_size, vocab)쌍으로 이루어져 있습니다.
    x = self.fc(output)

    # return x, state, attention_weights
    return x, state

In [16]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)
decoder2 = Decoder(vocab_tar_size2, embedding_dim, units, BATCH_SIZE)
decoder3 = Decoder(vocab_tar_size3, embedding_dim, units, BATCH_SIZE)

In [17]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [18]:
# 여기서 학습한 매개변수를 저장, optimizer/encoder/decoder
checkpoint_dir = '/content/drive/MyDrive/Colab Notebooks/training_checkpoints'
checkpoint_dir2 = '/content/drive/MyDrive/Colab Notebooks/training_checkpoints_2'
checkpoint_dir3 = '/content/drive/MyDrive/Colab Notebooks/training_checkpoints_3'

# checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)
checkpoint2 = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder2,
                                 decoder=decoder2)
checkpoint3 = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder3,
                                 decoder=decoder3)

In [19]:
def evaluate(sentence):
  # 어텐션 그래프
  # attention_plot = np.zeros((max_length_targ, max_length_inp))

  sentence = preprocess_sentence(sentence)

  # 문장, input 딕셔너리 출력 
  print ('sentence:', sentence)

  inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs2 = [inp_lang.word_index[i] for i in sentence.split(' ')]
  inputs2 = tf.keras.preprocessing.sequence.pad_sequences([inputs2],
                                                         maxlen=max_length_inp2,
                                                         padding='post')
  inputs3 = [inp_lang.word_index[i] for i in sentence.split(' ')]
  inputs3 = tf.keras.preprocessing.sequence.pad_sequences([inputs3],
                                                         maxlen=max_length_inp3,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)
  inputs2 = tf.convert_to_tensor(inputs2)
  inputs3 = tf.convert_to_tensor(inputs3)


  result = ''
  result2 = ''
  result3 = ''

  hidden = [tf.zeros((1, units))]
  hidden2 = [tf.zeros((1, units))]
  hidden3 = [tf.zeros((1, units))]

  enc_out, enc_hidden = encoder(inputs, hidden)
  enc_out2, enc_hidden2 = encoder2(inputs2, hidden2)
  enc_out3, enc_hidden3 = encoder3(inputs3, hidden3)


  dec_hidden = enc_hidden
  dec_hidden2 = enc_hidden2
  dec_hidden3 = enc_hidden3


  # Target 딕셔너리 출력
  dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

  for t in range(max(max_length_inp, max_length_inp2, max_length_targ3)):
    predictions, dec_hidden = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)
    predictions2, dec_hidden2 = decoder(dec_input,
                                                         dec_hidden2,
                                                         enc_out2)
    predictions3, dec_hidden3 = decoder(dec_input,
                                                         dec_hidden3,
                                                         enc_out3)
    
    # Ensemble - Soft Voting (다른 데이터를 가지고 했을 때)
    predictions_sum = tf.add(predictions[0], predictions2[0], predictions3[0])
    mean = tf.divide(predictions_sum, 3)
    voting_id = tf.argmax(mean).numpy()

    # tensorflow 에서 .numpy(): Tensor -> Numpy 로 바꿔주는 역할
    # predictions[0]  = tf.Tensor([-7.6920695 -7.9410057  3.7609155 ... ])
    
    result += targ_lang.index_word[voting_id] + ' '
    print('result: ', result)

    if targ_lang.index_word[voting_id] == '<end>':
      # return result, sentence, attention_plot
      return result, sentence

    # 예측된 ID를 모델에 다시 피드합니다.
    dec_input = tf.expand_dims([voting_id], 0)
    print("for 문 후 dec_input : ", dec_input)

  # for t in range(max_length_targ):
  #   predictions, dec_hidden, attention_weights = decoder(dec_input,
  #                                                        dec_hidden,
  #                                                        enc_out)

    # 나중에 어텐션 가중치를 시각화하기 위해 어텐션 가중치를 저장합니다.
    # attention_weights = tf.reshape(attention_weights, (-1, ))
    # attention_plot[t] = attention_weights.numpy()

#     predicted_id_list = []   # 제일 큰 확률 5개의 id 저장 리스트 (확률 값이 아니라 해당 아이디!!!!)
#     search_max_predictions = list(predictions[0])  # 큰 값 5개를 찾기 위해서 임시로 복사한 확률 분포 리스트
#     for i in range(5):
#         argmax_id = tf.argmax(search_max_predictions).numpy() # 확률 제일 큰 값 
#         predicted_id_list.append(argmax_id)
#         del search_max_predictions[argmax_id]  # 찾은 제일 큰 값 리스트에서 삭제 -> 다음 루프에서 그 다음으로 큰 값 찾기
    
#     # 예측된 5개의 id 에 해당하는 단어 출력
#     for i in predicted_id_list:
#         print(i, ' : ', targ_lang.index_word[i])
        
#     # 다음 나올 단어 입력
#     predicted_id = int(input("다음 단어의 ID 를 입력하세요 : "))
# #     predicted_id = max(predicted_id_list)
    
#     print('predicted_id_list : ', predicted_id_list)
    

    # result += targ_lang.index_word[predicted_id] + ' '
    # print('result: ', result)

    # if targ_lang.index_word[predicted_id] == '<end>':
    #   return result, sentence, attention_plot

    # # 예측된 ID를 모델에 다시 피드합니다.
    # dec_input = tf.expand_dims([predicted_id], 0)
    # print("for 문 후 dec_input : ", dec_input)

  # return result, sentence, attention_plot
  return result, sentence

In [20]:
def translate(sentence):
  result, sentence = evaluate(sentence)

  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

  # attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
#   plot_attention(attention_plot, sentence.split(' '), result.split(' '))

In [21]:
# checkpoint_dir내에 있는 최근 체크포인트(checkpoint)를 복원
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
checkpoint2.restore(tf.train.latest_checkpoint(checkpoint_dir2))
checkpoint3.restore(tf.train.latest_checkpoint(checkpoint_dir3))

In [28]:
translate(u'hace mucho frio aqui.')  # it s very cold here

sentence: <start> hace mucho frio aqui . <end>
result:  it 
for 문 후 dec_input :  tf.Tensor([[10]], shape=(1, 1), dtype=int32)
result:  it s 
for 문 후 dec_input :  tf.Tensor([[11]], shape=(1, 1), dtype=int32)
result:  it s very 
for 문 후 dec_input :  tf.Tensor([[48]], shape=(1, 1), dtype=int32)
result:  it s very cold 
for 문 후 dec_input :  tf.Tensor([[183]], shape=(1, 1), dtype=int32)
result:  it s very cold is 
for 문 후 dec_input :  tf.Tensor([[8]], shape=(1, 1), dtype=int32)
result:  it s very cold is good 
for 문 후 dec_input :  tf.Tensor([[69]], shape=(1, 1), dtype=int32)
result:  it s very cold is good early 
for 문 후 dec_input :  tf.Tensor([[287]], shape=(1, 1), dtype=int32)
result:  it s very cold is good early . 
for 문 후 dec_input :  tf.Tensor([[3]], shape=(1, 1), dtype=int32)
result:  it s very cold is good early . <end> 
Input: <start> hace mucho frio aqui . <end>
Predicted translation: it s very cold is good early . <end> 


In [23]:
translate(u'esta es mi vida.')  # this is my life

sentence: <start> esta es mi vida . <end>
result:  this 
for 문 후 dec_input :  tf.Tensor([[19]], shape=(1, 1), dtype=int32)
result:  this is 
for 문 후 dec_input :  tf.Tensor([[8]], shape=(1, 1), dtype=int32)
result:  this is my 
for 문 후 dec_input :  tf.Tensor([[21]], shape=(1, 1), dtype=int32)
result:  this is my life 
for 문 후 dec_input :  tf.Tensor([[189]], shape=(1, 1), dtype=int32)
result:  this is my life . 
for 문 후 dec_input :  tf.Tensor([[3]], shape=(1, 1), dtype=int32)
result:  this is my life . <end> 
Input: <start> esta es mi vida . <end>
Predicted translation: this is my life . <end> 


In [27]:
translate(u'¿todavia estan en casa?')  # Are you still at home?

sentence: <start> ¿ todavia estan en casa ? <end>
result:  are 
for 문 후 dec_input :  tf.Tensor([[24]], shape=(1, 1), dtype=int32)
result:  are you 
for 문 후 dec_input :  tf.Tensor([[6]], shape=(1, 1), dtype=int32)
result:  are you still 
for 문 후 dec_input :  tf.Tensor([[103]], shape=(1, 1), dtype=int32)
result:  are you still home 
for 문 후 dec_input :  tf.Tensor([[89]], shape=(1, 1), dtype=int32)
result:  are you still home ? 
for 문 후 dec_input :  tf.Tensor([[7]], shape=(1, 1), dtype=int32)
result:  are you still home ? <end> 
Input: <start> ¿ todavia estan en casa ? <end>
Predicted translation: are you still home ? <end> 


In [29]:
# 잘못된 번역
translate(u'trata de averiguarlo.')   # try to find out / try to figure out

sentence: <start> trata de averiguarlo . <end>
result:  try 
for 문 후 dec_input :  tf.Tensor([[125]], shape=(1, 1), dtype=int32)
result:  try to 
for 문 후 dec_input :  tf.Tensor([[15]], shape=(1, 1), dtype=int32)
result:  try to figure 
for 문 후 dec_input :  tf.Tensor([[891]], shape=(1, 1), dtype=int32)
result:  try to figure it 
for 문 후 dec_input :  tf.Tensor([[10]], shape=(1, 1), dtype=int32)
result:  try to figure it out 
for 문 후 dec_input :  tf.Tensor([[74]], shape=(1, 1), dtype=int32)
result:  try to figure it out of 
for 문 후 dec_input :  tf.Tensor([[59]], shape=(1, 1), dtype=int32)
result:  try to figure it out of us 
for 문 후 dec_input :  tf.Tensor([[81]], shape=(1, 1), dtype=int32)
result:  try to figure it out of us . 
for 문 후 dec_input :  tf.Tensor([[3]], shape=(1, 1), dtype=int32)
result:  try to figure it out of us . <end> 
Input: <start> trata de averiguarlo . <end>
Predicted translation: try to figure it out of us . <end> 


In [26]:
translate(u'Te quiero')   # I love you

sentence: <start> te quiero <end>
result:  i 
for 문 후 dec_input :  tf.Tensor([[4]], shape=(1, 1), dtype=int32)
result:  i m 
for 문 후 dec_input :  tf.Tensor([[18]], shape=(1, 1), dtype=int32)
result:  i m staying 
for 문 후 dec_input :  tf.Tensor([[1383]], shape=(1, 1), dtype=int32)
result:  i m staying . 
for 문 후 dec_input :  tf.Tensor([[3]], shape=(1, 1), dtype=int32)
result:  i m staying . <end> 
Input: <start> te quiero <end>
Predicted translation: i m staying . <end> 
